In [74]:
!pip install keras-tuner


In [75]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [76]:
fashion_mnist = keras.datasets.fashion_mnist

In [77]:
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

In [78]:
train_images = train_images/255.0 
test_images = test_images/255.0

In [79]:
train_images[0].shape

(28, 28)

In [80]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [81]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernal', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernal', values = [3,5]),
        activation='relu',
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax') #Output Layer         
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [82]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [83]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name='Mnist Fasshion')

INFO:tensorflow:Reloading Oracle from existing project output/Mnist Fasshion/oracle.json
INFO:tensorflow:Reloading Tuner from output/Mnist Fasshion/tuner0.json


In [84]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

INFO:tensorflow:Oracle triggered exit


In [85]:
model=tuner_search.get_best_models(num_models=1)[0]

In [86]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 112)       2912      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        48432     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               2602096   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 2,654,570
Trainable params: 2,654,570
Non-trainable params: 0
_________________________________________________________________
